# Modèle de Prédiction des Hospitalisations avec PyCaret

In [1]:
import numpy as np
from pycaret.regression import *
from sklearn.model_selection import train_test_split
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.cloud import bigquery
import os

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/antob/Documents/Arctusol/projet_wagon/projet_data_JBN/projet-jbn-data-le-wagon-533639ce801d.json"
        # Initialisation du client BigQuery
client = bigquery.Client()

query = """
SELECT * FROM projet-jbn-data-le-wagon.dbt_medical_analysis_join_total_morbidite.class_join_total_morbidite_sexe_population

"""

df_hospi = client.query(query).to_dataframe()


In [7]:
df_hospi_dpt_ens = df_hospi[(df_hospi["niveau"] == "Départements") & (df_hospi["sexe"] == "Ensemble")]
df_hospi_dpt_HF = df_hospi[(df_hospi["niveau"] == "Départements") & (df_hospi["sexe"] != "Ensemble")]
df_hospi_reg_ens = df_hospi[(df_hospi["niveau"] == "Régions") & (df_hospi["sexe"] == "Ensemble")]
df_hospi_reg_HF = df_hospi[(df_hospi["niveau"] == "Régions") & (df_hospi["sexe"] != "Ensemble")]

In [8]:
df_hospi_dpt_ens_yr = df_hospi_dpt_ens[df_hospi_dpt_ens["annee"].isin([2018, 2019,2020,2021, 2022])]
df_hospi_dpt_HF_yr =df_hospi_dpt_HF[df_hospi_dpt_HF["annee"].isin([2018, 2019,2020,2021, 2022])]
df_hospi_reg_ens_yr = df_hospi_reg_ens[df_hospi_reg_ens["annee"].isin([2018, 2019,2020, 2021, 2022])]
df_hospi_reg_HF_yr =df_hospi_reg_HF[df_hospi_reg_HF["annee"].isin([2018, 2019,2020, 2021, 2022])]

In [10]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])].copy()
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022].copy()

# Afficher les valeurs uniques
print("Régions disponibles:")
print(train_data['nom_region'].unique())
print("\nPathologies disponibles:")
print(train_data['nom_pathologie'].unique())

# Vérifier une combinaison qui existe
sample_data = train_data.groupby(['nom_region', 'nom_pathologie']).size().reset_index()
print("\nExemple de combinaison région/pathologie:")
print(sample_data.iloc[0])

# Utiliser cette combinaison
region = sample_data.iloc[0]['nom_region']
pathologie = sample_data.iloc[0]['nom_pathologie']

print(f"\nUtilisation de la combinaison:\nRégion: {region}\nPathologie: {pathologie}")

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
].sort_values('annee').copy()

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
].copy()

# Vérifier les données
print("\nNombre de lignes dans train_filtered:", len(train_filtered))
print("Données d'entraînement:")
print(train_filtered[['annee', 'nbr_hospi']])

Régions disponibles:
['Corse' 'Guyane' 'Mayotte' 'Bretagne' 'Normandie' 'Grand Est' 'Occitanie'
 'Guadeloupe' 'Martinique' 'La Réunion' 'Ile de France' 'Hauts-de-France'
 'Pays de la Loire' 'Nouvelle-Aquitaine' 'Centre-Val de Loire'
 'Auvergne-Rhône-Alpes' 'Bourgogne-Franche-Comté'
 "Provence-Alpes-Côte d'Azur"]

Pathologies disponibles:
['Maladies intestinales infectieuses' 'Tuberculose' 'Septicémies'
 'Mal.sexuellement transmissibles' 'Hépatites virales'
 'Maladies dues au V.I.H.' 'Tumeurs malignes' 'T.M. lèvre _bouche_pharynx'
 "Tumeur maligne de l'oesophage" "Tumeur maligne de l'estomac"
 'Tumeurs malignes colo' 'T.M.foie et voies biliaires intrahép'
 'Tumeur maligne du pancréas' 'Tumeur maligne du larynx'
 'T.M. trachée_bronches_poumon' 'Mélanome malin de la peau'
 'Tumeur maligne du sein' "Tumeur maligne de l'utérus"
 'Tumeur maligne de la prostate' 'Tumeur maligne de la vessie'
 'T.M. système nerveux central' 'Tumeur maligne de la thyroïde'
 'Leucémies' 'Tumeurs malignes seconda

In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [13]:
# Préparons les données avec une région et une pathologie spécifiques
region = 'Nouvelle-Aquitaine'
pathologie = 'Tumeurs malignes'

# Préparation des données d'entraînement et de test
train_data = df_hospi_reg_ens_yr[
    (df_hospi_reg_ens_yr['annee'].between(2018, 2021)) &
    (df_hospi_reg_ens_yr['nom_region'] == region) &
    (df_hospi_reg_ens_yr['nom_pathologie'] == pathologie)
].copy()

test_data = df_hospi_reg_ens_yr[
    (df_hospi_reg_ens_yr['annee'] == 2022) &
    (df_hospi_reg_ens_yr['nom_region'] == region) &
    (df_hospi_reg_ens_yr['nom_pathologie'] == pathologie)
].copy()

# Ajout des caractéristiques supplémentaires
train_data['Series'] = range(len(train_data))
train_data['annee_norm'] = (train_data['annee'] - train_data['annee'].min()) / (train_data['annee'].max() - train_data['annee'].min())

# Même chose pour les données de test
test_data['Series'] = range(len(test_data))
test_data['annee_norm'] = (test_data['annee'] - train_data['annee'].min()) / (train_data['annee'].max() - train_data['annee'].min())

# Configuration de PyCaret
reg = setup(
    data=train_data,
    test_data=test_data,
    target='nbr_hospi',
    fold_strategy='timeseries',
    numeric_features=['annee', 'Series', 'annee_norm'],
    fold=3,
    transform_target=True,
    session_id=123,
    verbose=False,
    data_split_shuffle=False,
    fold_shuffle=False,
    categorical_features=['nom_region', 'nom_pathologie']
)

# Création et comparaison des modèles
best_model = compare_models(n_select=1)

# Créer un DataFrame complet pour les prédictions futures
future_dates = pd.DataFrame({
    'niveau': ['Régions'] * 3,
    'sexe': ['Ensemble'] * 3,
    'year': [f'{year}-12-31' for year in [2023, 2024, 2025]],
    'annee': [2023, 2024, 2025],
    'pathologie': [pathologie] * 3,
    'code_pathologie': train_data['code_pathologie'].iloc[0],
    'nom_pathologie': [pathologie] * 3,
    'region': [region] * 3,
    'code_region': train_data['code_region'].iloc[0],
    'nom_region': [region] * 3,
    'Series': range(len(train_data) + len(test_data), len(train_data) + len(test_data) + 3),
    'annee_norm': [(year - train_data['annee'].min()) / (train_data['annee'].max() - train_data['annee'].min()) 
                   for year in [2023, 2024, 2025]]
})

# Copier toutes les autres colonnes de la dernière ligne des données d'entraînement
for col in train_data.columns:
    if col not in future_dates.columns:
        future_dates[col] = train_data[col].iloc[-1]

# Faire les prédictions
predictions = predict_model(best_model, data=future_dates)
print("\nPrédictions pour les années futures:")
print(predictions[['annee', 'prediction_label']])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dummy,Dummy Regressor,1207.3627,3279203.1546,1207.3627,nan,0.0166,0.0170,0.0333


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Dummy Regressor,155.5370,24191.7520,155.5370,0.0000,0.0021,0.0021



Prédictions pour les années futures:
   annee  prediction_label
0   2023       73306.46302
1   2024       73306.46302
2   2025       73306.46302


In [12]:
# Affichons la structure actuelle des données
print("Colonnes disponibles dans df_hospi_reg_ens_yr:")
print(df_hospi_reg_ens_yr.columns.tolist())

# Affichons également un exemple des données
print("\nAperçu des données:")
print(df_hospi_reg_ens_yr.head())

Colonnes disponibles dans df_hospi_reg_ens_yr:
['niveau', 'cle_unique', 'sexe', 'year', 'annee', 'pathologie', 'code_pathologie', 'nom_pathologie', 'region', 'code_region', 'nom_region', 'nbr_hospi', 'evolution_nbr_hospi', 'evolution_percent_nbr_hospi', 'hospi_prog_24h', 'hospi_autres_24h', 'hospi_total_24h', 'hospi_1J', 'hospi_2J', 'hospi_3J', 'hospi_4J', 'hospi_5J', 'hospi_6J', 'hospi_7J', 'hospi_8J', 'hospi_9J', 'hospi_10J_19J', 'hospi_20J_29J', 'hospi_30J', 'hospi_total_jj', 'total_hospi', 'AVG_duree_hospi', 'evolution_hospi_total_24h', 'evolution_percent_hospi_total_24h', 'evolution_hospi_total_jj', 'evolution_percent_hospi_total_jj', 'evolution_total_hospi', 'evolution_percent_total_hospi', 'evolution_AVG_duree_hospi', 'evolution_percent_AVG_duree_hospi', 'tranche_age_0_1', 'tranche_age_1_4', 'tranche_age_5_14', 'tranche_age_15_24', 'tranche_age_25_34', 'tranche_age_35_44', 'tranche_age_45_54', 'tranche_age_55_64', 'tranche_age_65_74', 'tranche_age_75_84', 'tranche_age_85_et_plus

In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [ ]:
# Préparation des données d'entraînement
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019, 2020, 2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection d'une région et pathologie spécifique pour le test
region = 'Nouvelle-Aquitaine'
pathologie = 'Maladies de l\'appareil circulatoire'

# Filtrer les données pour une région et pathologie spécifique
train_filtered = train_data[
    (train_data['nom_region'] == region) & 
    (train_data['nom_pathologie'] == pathologie)
]

test_filtered = test_data[
    (test_data['nom_region'] == region) & 
    (test_data['nom_pathologie'] == pathologie)
]

# Créer une séquence numérique pour l'ordre temporel
train_filtered['Series'] = np.arange(1, len(train_filtered) + 1)
test_filtered['Series'] = np.arange(len(train_filtered) + 1, len(train_filtered) + len(test_filtered) + 1)


In [ ]:

# Initialisation de l'environnement PyCaret avec stratégie temporelle
reg = setup(
    data=train_filtered,
    test_data=test_filtered,
    target='nbr_hospi',
    fold_strategy='timeseries',  # Utiliser la stratégie de séries temporelles
    numeric_features=['annee', 'Series'],
    fold=5,
    transform_target=True,
    session_id=123,
    verbose=False,
    data_split_shuffle=False,  # Ajouté : Ne pas mélanger les données
    fold_shuffle=False 
)

# Comparaison des modèles
best_model = compare_models(sort='MAE')

# Finaliser le modèle (entraîner sur toutes les données)
final_model = finalize_model(best_model)

# Créer le dataset pour les prédictions futures
future_years = [2023, 2024, 2025]
future_df = pd.DataFrame({
    'annee': future_years,
    'Series': np.arange(len(train_filtered) + len(test_filtered) + 1, 
                       len(train_filtered) + len(test_filtered) + len(future_years) + 1),
    'nom_region': region,
    'nom_pathologie': pathologie
})

# Faire les prédictions
predictions_future = predict_model(final_model, data=future_df)
print("\nPrédictions pour les années futures:")
print(predictions_future[['annee', 'nom_region', 'nom_pathologie', 'prediction_label']])

# Afficher les résultats complets
all_data = pd.concat([
    train_filtered[['annee', 'nbr_hospi']],
    test_filtered[['annee', 'nbr_hospi']],
    predictions_future[['annee', 'prediction_label']].rename(columns={'prediction_label': 'nbr_hospi'})
])

print("\nHistorique et prédictions complètes:")
print(all_data.sort_values('annee'))

In [ ]:
    """'hospi_prog_24h', 'hospi_autres_24h', 'hospi_total_24h',
    'hospi_1J', 'hospi_2J', 'hospi_3J', 'hospi_4J', 'hospi_5J',
    'hospi_6J', 'hospi_7J', 'hospi_8J', 'hospi_9J',
    'hospi_10J_19J', 'hospi_20J_29J', 'hospi_30J',
        'tranche_age_0_1', 'tranche_age_1_4', 'tranche_age_5_14',
    'tranche_age_15_24', 'tranche_age_25_34', 'tranche_age_35_44',
    'tranche_age_45_54', 'tranche_age_55_64', 'tranche_age_65_74',
    'tranche_age_75_84', 'tranche_age_85_et_plus'"""

In [6]:
# Préparation des données d'entraînement (2018-2019)
train_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'].isin([2018, 2019,2020,2021])]
test_data = df_hospi_reg_ens_yr[df_hospi_reg_ens_yr['annee'] == 2022]

# Sélection des variables pertinentes
features = [
    'annee', 'nom_pathologie', 'nom_region'

]

# Préparation des données pour PyCaret
train_data = train_data[features + ['nbr_hospi']]
test_data = test_data[features + ['nbr_hospi']]

In [ ]:
# Initialisation de l'environnement PyCaret
reg = setup(
    data=train_data,
    target='nbr_hospi',
    session_id=123,
    normalize=True,
    transformation=True,
    fold=5,  # 5-fold cross validation
)

In [ ]:
# Comparaison des modèles
best_model = compare_models(  # On exclut les modèles moins pertinents pour ce cas
    n_select=1  # Sélectionner le meilleur modèle
)

In [9]:
# Tuning du meilleur modèle
#tuned_model = tune_model(best_model)

In [ ]:
# Analyse du MAE en contexte
print("Statistiques du nombre d'hospitalisations:")
print(train_data['nbr_hospi'].describe())

# Calcul du MAE relatif
mae_relatif = 211 / train_data['nbr_hospi'].mean() * 100
print(f"\nMAE en pourcentage de la moyenne: {mae_relatif:.2f}%")

# Distribution des erreurs
plt.figure(figsize=(10, 6))
erreurs = abs(predictions['nbr_hospi'] - predictions['prediction_label'])
sns.histplot(data=erreurs[erreurs < erreurs.quantile(0.95)], bins=50)
plt.axvline(x=211, color='r', linestyle='--', label='MAE')
plt.title('Distribution des erreurs absolues')
plt.xlabel('Erreur absolue (nombre d\'hospitalisations)')
plt.legend()
plt.show()

In [ ]:
# Création de données de test pour 3 années futures
future_data = pd.DataFrame([
    {'annee': 2023, 'nom_pathologie': 'Maladies de l\'appareil circulatoire', 'nom_region': 'Île-de-France'},
    {'annee': 2024, 'nom_pathologie': 'Maladies de l\'appareil circulatoire', 'nom_region': 'Île-de-France'},
    {'annee': 2025, 'nom_pathologie': 'Maladies de l\'appareil circulatoire', 'nom_region': 'Île-de-France'}
])

# Faire les prédictions
predictions_futures = predict_model(best_model, data=future_data)
print("Prédictions pour les années futures:")
print(predictions_futures[['annee', 'nom_region', 'nom_pathologie', 'prediction_label']])


In [ ]:
# Évaluation sur les données de test (2022)
predictions = predict_model(best_model, data=test_data)
print("Performance sur les données de test (2022):")
print(predictions.head())

In [ ]:
# Calcul des métriques d'erreur pour les prédictions
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Extraire les vraies valeurs et les prédictions
y_true = predictions['nbr_hospi']
y_pred = predictions['prediction_label']

# Calcul des métriques
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"Métriques sur les données de test (2022):")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

# Visualisation des prédictions vs réalité
plt.figure(figsize=(10, 6))
plt.scatter(y_true, y_pred, alpha=0.5)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
plt.xlabel('Valeurs réelles')
plt.ylabel('Prédictions')
plt.title('Prédictions vs Valeurs réelles')
plt.show()

# Calcul et affichage des erreurs relatives
predictions['erreur_relative'] = abs(predictions['nbr_hospi'] - predictions['prediction_label']) / predictions['nbr_hospi'] * 100
print("\nStatistiques des erreurs relatives (%):")
print(predictions['erreur_relative'].describe())

In [ ]:
save_model(best_model, 'best_model')